## Logistic Regression Consulting Project

In [1]:
import findspark

In [2]:
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('customer_churn').getOrCreate()

In [5]:
from pyspark.ml.classification import LogisticRegression

In [6]:
customer_data = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)

In [8]:
customer_data.head(1)

[Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1)]

In [10]:
for item in customer_data.head(1)[0]:
    print(item)

Cameron Williams
42.0
11066.8
0
7.22
8.0
2013-08-30 07:00:40
10265 Elizabeth Mission Barkerburgh, AK 89518
Harvey LLC
1


In [11]:
customer_data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



W tym momencie nalezy podjąć decyzję o tym który czynniki mogą mieć wpływ na to czy klient zrezygnuje z usług firmy. Używamy metody describe() aby zweryfikować czy mamy wszystkie wartości we wszystkich kolumnach. Używamy własciwości columns aby sprawdzić nazwy poszczególnych cech.

In [12]:
customer_data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [14]:
customer_data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [16]:
from pyspark.ml.feature import VectorAssembler

In [18]:
assembler = VectorAssembler(inputCols=['Age',
                                    'Total_Purchase',
                                     'Account_Manager',
                                     'Years',
                                     'Num_Sites'], outputCol='features')

In [24]:
output = assembler.transform(customer_data)

In [25]:
final_data = output.select('features', 'churn')

In [26]:
train_churn, test_churn = final_data.randomSplit([0.7, 0.3])

In [27]:
from pyspark.ml.classification import LogisticRegression

In [28]:
lr_churn = LogisticRegression(labelCol='churn')

In [29]:
fitted_churn_model = lr_churn.fit(train_churn)

In [31]:
traininng_sum = fitted_churn_model.summary

In [32]:
traininng_sum.predictions.describe().show()

+-------+------------------+-------------------+
|summary|             churn|         prediction|
+-------+------------------+-------------------+
|  count|               598|                598|
|   mean|0.1688963210702341|0.13210702341137123|
| stddev|0.3749739724566811|0.33889055651835864|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



In [33]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [34]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [35]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8787.39,1.0...|    1|[0.18327379717179...|[0.54569062787385...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.21432726627374...|[0.77106371325155...|       0.0|
|[28.0,11128.95,1....|    0|[4.22011530070436...|[0.98551592193031...|       0.0|
|[28.0,11204.23,0....|    0|[1.57145606286147...|[0.82799108248027...|       0.0|
|[29.0,9378.24,0.0...|    0|[4.76509693532625...|[0.99154994973969...|       0.0|
|[29.0,11274.46,1....|    0|[4.59068067086454...|[0.98995595639399...|       0.0|
|[30.0,6744.87,0.0...|    0|[3.29810626585056...|[0.96436378734238...|       0.0|
|[30.0,8403.78,1.0...|    0|[6.09553466285986...|[0.99775215889920...|       0.0|
|[30.0,8677.28,1.0...|    0|[4.14923332568900...|[0.98446852513421...|       0.0|
|[30.0,12788.37,

In [36]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='churn')

In [37]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [38]:
auc

0.782407033959829

In [39]:
final_lr_model = lr_churn.fit(final_data)

In [40]:
new_customers = spark.read.csv('new_customers.csv', inferSchema=True, header=True)

In [41]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [42]:
test_new_customers = assembler.transform(new_customers)

In [43]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [44]:
final_results = final_lr_model.transform(test_new_customers)

In [45]:
final_results.show()

+--------------+----+--------------+---------------+-----+---------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|        Onboard_date|            Location|         Company|            features|       rawPrediction|         probability|prediction|
+--------------+----+--------------+---------------+-----+---------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:...|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,1.0...|[2.22168705251440...|[0.90218018099704...|       0.0|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:...|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,1.0...|[-6.2207530595020...|[0.00198380445828...|  

In [46]:
final_results.select('Company', 'prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



In [47]:
test_new_customers.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|         Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                   6|               6|
|   mean|         null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|                null|            null|
| stddev|         null| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|                null|            null|
|    min|Andrew Mccall|              22.0|            100.0|          